In [1]:
import json
import pandas as pd
from pandas import json_normalize
from pprint import pprint

In [2]:
import sqlite3 
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [3]:
connection = create_connection(r"C:\Users\Irene\lighthouse-data-notes\week2\miniproject2\data\test.sqlite")

Connection to SQLite DB successful


# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [4]:
with open(r'C:\Users\Irene\lighthouse-data-notes\week2\miniproject2\data\petstore_fs.json') as f:
    fs = json.load(f)
pprint (fs)

{'context': {'geo_bounds': {'circle': {'center': {'latitude': 39.703014,
                                                  'longitude': -105.078792},
                                       'radius': 1000}}},
 'results': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pet_store_',
                                       'suffix': '.png'},
                              'id': 17110,
                              'name': 'Pet Supplies Store'}],
              'chains': [{'id': '40ac1d6f-112b-4664-be82-8f563190b1c8',
                          'name': 'Petco'}],
              'distance': 621,
              'fsq_id': '4c8c0541ed3ab60c04ea6321',
              'geocodes': {'main': {'latitude': 39.707519,
                                    'longitude': -105.08336},
                           'roof': {'latitude': 39.707519,
                                    'longitude': -105.08336}},
              'link': '/v3/places/4c8c0541ed3ab60c04ea6321',
              'lo

Parse through the response to get the POI details you want (rating, name, location, etc)

In [5]:
from pandas import json_normalize
json_normalize(fs)
results = json_normalize(fs, record_path='results')
categories = json_normalize(fs['results'], record_path='categories', errors='ignore')

results.rename(columns = {'name':'petstore_name', 'location.formatted_address':'location_address'}, 
               inplace = True)
categories = json_normalize(fs['results'], record_path='categories', 
                            meta=['petstore_name','location_address'], errors='ignore')

fs_pet_results = results[['petstore_name','location_address', 'distance']]
fs_pet_results.head()

,petstore_name,location_address,distance
0,Petco,"475 S Wadsworth Blvd, Lakewood, CO 80226",621


In [6]:
categorylist = []
for i in range(len(results['petstore_name'])):
    categorylist.append(fs['results'][i]['categories'][0]['name'])

cat_list = pd.DataFrame(categorylist)
cat_list.columns = ['category']

Put your parsed results into a DataFrame

In [7]:
pets_fs = pd.concat([fs_pet_results, cat_list], axis=1, join='inner')
pets_fs.sort_values(by = 'distance')

,petstore_name,location_address,distance,category
0,Petco,"475 S Wadsworth Blvd, Lakewood, CO 80226",621,Pet Supplies Store


In [8]:
pets_fs['petstore_name'] = pets_fs['petstore_name'].astype('str')
pets_fs['location_address'] = pets_fs['location_address'].astype('str')


# Yelp

Send a request with the same location paramaters (location, radius, etc)

In [9]:
with open(r'C:\Users\Irene\lighthouse-data-notes\week2\miniproject2\data\petstore_yelp.json') as y:
    yelp = json.load(y)
pprint (yelp)

{'businesses': [{'alias': 'petco-lakewood',
                 'categories': [{'alias': 'petstore', 'title': 'Pet Stores'},
                                {'alias': 'groomer', 'title': 'Pet Groomers'},
                                {'alias': 'pet_training',
                                 'title': 'Pet Training'}],
                 'coordinates': {'latitude': 39.7075, 'longitude': -105.0815},
                 'display_phone': '(303) 985-0050',
                 'distance': 549.9889720383354,
                 'id': '08DAVrQimvbBIzm29xMv3w',
                 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/pJ9zHU4IsfgswfMBH-mz0A/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '475 S Wadsworth',
                              'address2': '',
                              'address3': '',
                              'city': 'Lakewood',
                              'country': 'US',
                              'display_address': ['475 S Wadswor

Parse through your result and get POI details

In [10]:
from pandas import json_normalize
json_normalize(yelp)
results = json_normalize(yelp, record_path='businesses')
results.rename(columns = {'name':'petstore_name', 'location.display_address':'location_address'}, 
               inplace = True)
yelp_pet = results[['petstore_name','location_address','distance','rating','review_count']]
yelp_pet

,petstore_name,location_address,distance,rating,review_count
0,Petco,"[475 S Wadsworth, Lakewood, CO 80226]",549.988972,2.5,58


In [11]:
categorylist = []
for i in range(len(results['petstore_name'])):
    categorylist.append(yelp['businesses'][i]['categories'][0]['title'])

cat_list = pd.DataFrame(categorylist)
cat_list.columns = ['category']

Put your parsed results into a DataFrame

In [12]:
pets_yelp = pd.concat([yelp_pet, cat_list], axis=1, join='inner')
pets_yelp['distance'] = round(pets_yelp['distance'])
pets_yelp.sort_values(by = 'distance')


,petstore_name,location_address,distance,rating,review_count,category
0,Petco,"[475 S Wadsworth, Lakewood, CO 80226]",550.0,2.5,58,Pet Stores


In [13]:
pets_yelp['petstore_name'] = pets_yelp['petstore_name'].astype('str')
pets_yelp['location_address'] = pets_yelp['location_address'].astype('str')


# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [14]:
pets_fs.to_sql(name='pets_fs', con=connection, if_exists='replace', index=False)
df_pets_fs = pd.read_sql('select * from pets_fs', connection)
df_pets_fs

,petstore_name,location_address,distance,category
0,Petco,"475 S Wadsworth Blvd, Lakewood, CO 80226",621,Pet Supplies Store


In [16]:
pets_yelp.to_sql(name='pets_yelp', con=connection, if_exists='replace', index=False)
df_pets_yelp = pd.read_sql('select * from pets_yelp', connection)
df_pets_yelp

,petstore_name,location_address,distance,rating,review_count,category
0,Petco,"['475 S Wadsworth', 'Lakewood, CO 80226']",550.0,2.5,58,Pet Stores


Get the top 10 pet store according to their rating

- There is only 1 pet store in the searched area